In [ ]:
# 1. Import libraries and load the datasets
import pandas as pd

# Load the data into DataFrames
sd_df = pd.read_csv('unigrams.csv', sep=';', encoding='utf-8')
bigrams_df = pd.read_csv('bigrams.csv', sep=';', encoding='utf-8')
trigrams_df = pd.read_csv('trigrams.csv', sep=';', encoding='utf-8')

In [ ]:
# 2. Create the SegnaliDisc dictionary
SegnaliDisc = {}

# Populate SegnaliDisc with unigrams
for _, row in sd_df.iterrows():
    word = row['word']
    frequency = row['Frequency']
    SegnaliDisc[word] = {
        'frequency': frequency,
        'ngrams': {}
    }

In [ ]:
# 3. Add bigrams to SegnaliDisc
for _, row in bigrams_df.iterrows():
    bigram = row['word']
    bigram_frequency = row['Frequency']
    word1, word2 = bigram.split()

    added_to_sd = False
    for word in sd_df['word']:
        if word in [word1, word2]:
            SegnaliDisc[word]['ngrams'][bigram] = bigram_frequency
            added_to_sd = True

    if not added_to_sd:
        SegnaliDisc[bigram] = {
            'frequency': bigram_frequency,
            'ngrams': {}
        }

In [ ]:
# 4. Add trigrams to SegnaliDisc
for _, row in trigrams_df.iterrows():
    trigram = row['word']
    trigram_frequency = row['Frequency']
    word1, word2, word3 = trigram.split()
    trigram_words = trigram.split()

    added_to_bi = False
    for bigram, bigram_data in SegnaliDisc.items():
        if isinstance(bigram, str) and len(bigram.split()) == 2:
            b_word1, b_word2 = bigram.split()
            if b_word1 in trigram_words and b_word2 in trigram_words:
                b_index1 = trigram_words.index(b_word1)
                b_index2 = trigram_words.index(b_word2)
                if b_index1 < b_index2:
                    SegnaliDisc[bigram]['ngrams'][trigram] = trigram_frequency
                    added_to_bi = True

    added_to_sd_tri = False
    for word in sd_df['word']:
        if word in [word1, word2, word3]:
            SegnaliDisc[word]['ngrams'][trigram] = trigram_frequency
            added_to_sd_tri = True

    if not added_to_bi and not added_to_sd_tri:
        SegnaliDisc[trigram] = {
            'frequency': trigram_frequency,
            'ngrams': {}
        }

In [ ]:
# 5. Build the final DataFrame
final_data = []
for key, value in SegnaliDisc.items():
    row = {'SegnaleDiscorsivo': key, 'frequenza': value['frequency']}
    ngram_columns = list(value['ngrams'].keys())
    ngram_frequencies = list(value['ngrams'].values())

    for i, (ngram, freq) in enumerate(zip(ngram_columns, ngram_frequencies)):
        row[f'ngram{i + 1}'] = ngram
        row[f'ngram{i + 1}_freq'] = freq

    for i in range(len(ngram_columns), 3):
        row[f'ngram{i + 1}'] = pd.NA
        row[f'ngram{i + 1}_freq'] = pd.NA

    final_data.append(row)

final_df = pd.DataFrame(final_data)

In [ ]:
# 6. Save and display the result
final_df.to_csv('Discourse_markers_stats.csv', encoding='utf-8', index=False)
print(final_df)